In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product, chain, repeat
import copy

In [45]:
def genCombinations(countList, chooseStim='345678', difficultyRange=4):
    smallestStim = int(chooseStim[0])
    temp = [] # 得到所有分數*數量的list
    for i, count in enumerate(countList):
        i = i + smallestStim
        temp.append(np.repeat(i,count))
        
    ratings = [] # 將所有刺激給編號 (分數_編號)
    for i in range(len(temp)):
        stimRating = temp[i]
        ratings.append([str(j)+'_'+str(i+1) for i,j in enumerate(stimRating)])

    diffDict = {i+1: [] for i in range(difficultyRange)} # 組合難度分類
    for c in list(combinations(ratings,2)):
        diff = abs(int(c[1][0].split('_')[0])-int(c[0][0].split('_')[0]))
        if diff in diffDict.keys():
            diffDict[diff].append(c)
        
    AllCombinations = [] # 創建所有組合
    diffCombinationsDict = {i+1: [] for i in range(difficultyRange)} # 創建所有組合並以難度分類
    for diff, comb in diffDict.items():
        for stims in comb:
            Trial = list(product(stims[0], stims[1]))
            AllCombinations += Trial
            diffCombinationsDict[diff] += Trial
            
    return diffCombinationsDict, AllCombinations, ratings

In [189]:
def getStim(diffCombinationsDict, quotaDF, needCounts=None):
    
    removeUsedStim = copy.deepcopy(diffCombinationsDict)
    if needCounts == None:
        needCounts = {1: 160, 2: 120, 3: 80, 4: 40} #[150, 120, 90, 60]
        
    def checkQuota(stim):
        if (quotaDF[quotaDF['Name']==stim[0]]['NowCounts'].values[0]==0) or \
            (quotaDF[quotaDF['Name']==stim[1]]['NowCounts'].values[0]==0):
            return False
        return True
    global failed
    
    stimDict = {i+1: [] for i in range(4)}
    for diff, comb in list(diffCombinationsDict.items())[::-1]:
        for i in range(len(comb)):
            if len(stimDict[diff]) == needCounts[diff]:
                break
            index = np.random.randint(len(comb))
            toAdd = comb[index]
            diffCombinationsDict[diff].pop(index) # 有抽出來過就丟掉
            if checkQuota(toAdd):
                stimDict[diff].append(toAdd)
                removeUsedStim[diff].remove(toAdd)
                for img in toAdd:
                    updateIndex = quotaDF[quotaDF['Name']==img].index[0]
                    quotaDF.at[updateIndex, 'NowCounts'] -= 1
            else:
                failed+=1
            
    return stimDict, removeUsedStim, diffCombinationsDict

In [235]:
failed = 0
def createFirstStim(NowCounts, allcounts):
    print(NowCounts)
    diffCombinationsDict, AllCombinations, ratings = genCombinations(allcounts) # [6,8,10,10,8,12]
    allratings = [item for rating in ratings for item in rating]
    quotaDF = pd.DataFrame({'Name':allratings, 'NowCounts': NowCounts})
    global failed
    failed = 0
    stimDict, removeUsedStim, _ = getStim(diffCombinationsDict, quotaDF)
    print(len([item for rating in stimDict.values() for item in rating]))
    return stimDict, removeUsedStim, allratings

def createSecondStim(NowCounts, prevDiffComDict, allcounts, allratings):
    print(NowCounts)
    quotaDF = pd.DataFrame({'Name':allratings, 'NowCounts': NowCounts})
    global failed
    failed = 0
    stimDict, removeUsedStim, _ = getStim(prevDiffComDict, quotaDF)
    print(len([item for rating in stimDict.values() for item in rating]))
    return stimDict, removeUsedStim

In [243]:
def makeStim(allcounts):
    NowCounts=5
    while True:
        stimDict1, newDiffCombination, allratings = createFirstStim(NowCounts, allcounts)
        total = len([item for rating in stimDict1.values() for item in rating])
        if total == 400:
            break
        else:
            NowCounts+=1
    NowCounts1 = NowCounts
    NowCounts=5
    while True:
        diffComcopy = copy.deepcopy(newDiffCombination)
        stimDict2, removeUsedStim = createSecondStim(NowCounts, diffComcopy, allcounts, allratings)
        total = len([item for rating in stimDict2.values() for item in rating])
        if total == 400:
            break
        else:
            NowCounts+=1
    NowCounts2 = NowCounts
    
    return stimDict1, stimDict2, NowCounts1, NowCounts2

In [264]:
a, b, _, _ = makeStim([50,30,20,20,16,16,16,16])

5
400
5
400


In [31]:
stimOccur = []
for ind in range(4):
    stimOccur += a[ind]
stimOccur2 = []
for ind in range(4):
    stimOccur2 += b[ind]

In [32]:
stimOccur = set(stimOccur)
stimOccur2 = set(stimOccur2)

In [263]:
new = stimOccur.union(stimOccur2)
print(len(stimOccur))
print(len(stimOccur2))
print(len(new))

NameError: name 'stimOccur' is not defined

In [ ]:
hard = stimList1[0]
highLowList = [0 for i in range(4)]
for stim in hard:
    first = int(stim[0].split('_')[0])
    second = int(stim[1].split('_')[0])
    match first:
        case 5:
            highLowList[0]+=1
        case 6:
            highLowList[1]+=1
        case 7:
            highLowList[2]+=1
        case 8:
            highLowList[3]+=1
print(highLowList)

In [ ]:
unique, counts = np.unique(stimOccur1, return_counts=True)
stim1df = pd.DataFrame({'Name': unique, 'counts': counts})
stim1df[stim1df.counts < 7].reset_index()

In [ ]:
unique, counts = np.unique(stimOccur2, return_counts=True)
stim2df = pd.DataFrame({'Name': unique, 'counts': counts})
stim2df[stim2df.counts < 7].reset_index()